In [11]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence
import torch.nn as nn
import torch.nn.functional as F

In [6]:
a = torch.rand((3, 2, 3))
b = torch.rand((3, 3))
a, b

(tensor([[[0.0407, 0.3822, 0.1209],
          [0.5260, 0.1311, 0.5091]],
 
         [[0.2540, 0.2224, 0.1980],
          [0.6845, 0.7385, 0.2483]],
 
         [[0.6656, 0.7685, 0.0595],
          [0.4402, 0.4833, 0.8783]]]),
 tensor([[0.2021, 0.1006, 0.8287],
         [0.6925, 0.2373, 0.7721],
         [0.2806, 0.6980, 0.7132]]))

In [7]:
a + b.unsqueeze(1)

tensor([[[0.2428, 0.4828, 0.9496],
         [0.7281, 0.2317, 1.3379]],

        [[0.9464, 0.4597, 0.9701],
         [1.3770, 0.9758, 1.0204]],

        [[0.9462, 1.4665, 0.7727],
         [0.7208, 1.1813, 1.5915]]])

In [8]:
a[[1, 2]]

tensor([[[0.2540, 0.2224, 0.1980],
         [0.6845, 0.7385, 0.2483]],

        [[0.6656, 0.7685, 0.0595],
         [0.4402, 0.4833, 0.8783]]])

In [10]:
x = torch.tensor([1, 2, 3])
torch.cat([x, torch.tensor([1])])

tensor([1, 2, 3, 1])

In [5]:
np_mask = np.triu(np.ones((1, 4, 4)),
    k=1).astype('uint8') == 0
np_mask

array([[[ True, False, False, False],
        [ True,  True, False, False],
        [ True,  True,  True, False],
        [ True,  True,  True,  True]]])

In [50]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence
import torch.nn as nn

class MyData(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    seq_len = [s.size(0) for s in data]
    data = pad_sequence(data, batch_first=True).float()    
    data = data.unsqueeze(-1)
    data = pack_padded_sequence(data, seq_len, batch_first=True, enforce_sorted=False)
    return data

b = torch.tensor([1,2,3,4])
a = torch.tensor([5,6,7])
c = torch.tensor([7,8])
d = torch.tensor([9])
train_x = [a, b]

data = MyData(train_x)
data_loader = DataLoader(data, batch_size=2, shuffle=True, collate_fn=collate_fn)
batch_x = iter(data_loader).next()

rnn = nn.LSTM(1, 4, 1, batch_first=True)
h0 = torch.rand(1, 2, 4).float()
c0 = torch.rand(1, 2, 4).float()
out, (h1, c1) = rnn(batch_x, (h0, c0))
out

PackedSequence(data=tensor([[0.0997, 0.1950, 0.0290, 0.1128],
        [0.3349, 0.0675, 0.0088, 0.0861],
        [0.1630, 0.1847, 0.0133, 0.0808],
        [0.4957, 0.0481, 0.0120, 0.0508],
        [0.2692, 0.1285, 0.0189, 0.0660],
        [0.6363, 0.0183, 0.0076, 0.0274],
        [0.3934, 0.0808, 0.0200, 0.0556]], grad_fn=<CatBackward0>), batch_sizes=tensor([2, 2, 2, 1]), sorted_indices=tensor([0, 1]), unsorted_indices=tensor([0, 1]))

In [28]:
embedding_space = torch.rand([138, 1])
cluster_encode = F.one_hot(torch.tensor(3), num_classes=10)
new_feature = torch.cat([torch.rand([128]), cluster_encode])
embedding_space = torch.cat([embedding_space, new_feature.unsqueeze(1)], dim=1)
new_feature.shape, embedding_space.shape

(torch.Size([138]), torch.Size([138, 2]))

In [33]:
embedding_space[:, 1] = torch.zeros((138, 1))
embedding_space

RuntimeError: expand(torch.FloatTensor{[138, 1]}, size=[138]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [42]:
from tianshou.data import Batch
x = Batch(a=[1, 2, 3, 4], b=Batch())
y = Batch(a=Batch(), b=[5, 8])
#x[np.array([0, 1])] = y
x[3]

Batch(
    a: 4,
    b: Batch(),
)

In [56]:
different_obs = Batch(obs=torch.rand((1, 1, 5)), rwd=(torch.rand((1, 1))))
different_obs

Batch(
    obs: tensor([[[0.9264, 0.2640, 0.8176, 0.8163, 0.6387]]]),
    rwd: tensor([[0.8509]]),
)

In [59]:
a = np.array([0, 1, 2])
a[[1, 2]][[1, 0]]

array([2, 1])